In [24]:
import json
import re

# Loading gold standard

In [5]:
ont_name = "ont_4_book"

In [13]:
id_to_gt = dict()
with open(f"data/ground_truth/{ont_name}_ground_truth_v1.jsonl") as in_file:
    for line in in_file:
        data = json.loads(line)
        id_to_gt[data['id']] = data

In [14]:
data

{'id': 'ont_4_book_test_550',
 'sent': "Billiards at Half-Past Nine focuses on the Faehmel family's history, from the end of the 19th century, until that day; it largely reflects the opposition of the author (who won the Nobel Prize for Literature in 1972) had to the period of Nazism, as well as his aversion to war in general.",
 'triples': [{'sub': 'Billiards at Half-Past Nine',
   'rel': 'depicts',
   'obj': 'Family'}]}

# Loading system output

In [10]:
id_to_sys = dict()
with open(f"prompts/{ont_name}_llm_responses.jsonl") as in_file:
    for line in in_file:
        data = json.loads(line)
        id_to_sys[data['id']] = data

In [11]:
data

{'id': 'ont_4_book_test_550',
 'response': 'author(Billiards at Half-Past Nine,Kundera)\npublication\\_date(Billiards at Half-Past Nine,1978)\nillustrator(Billiards at Half-Past Nine,)\nnarrative\\_location(Billiards at Half-Past Nine,Czechoslovakia)\ngenre(Billiards at Half-Past Nine,literary genre)\nlanguage\\_of\\_work\\_or\\_name(Billiards at Half-Past Nine,Czech)\ndepicts(Billiards at Half-Past Nine,Nazism)\ndepicts(Billiards at Half-Past Nine,war)',
 'triples': [['Billiards at Half-Past Nine', 'author', 'Kundera'],
  ['Billiards at Half-Past Nine', '_date', '1978'],
  ['Billiards at Half-Past Nine', '_location', 'Czechoslovakia'],
  ['Billiards at Half-Past Nine', 'genre', 'literary genre'],
  ['Billiards at Half-Past Nine', '_name', 'Czech'],
  ['Billiards at Half-Past Nine', 'depicts', 'Nazism'],
  ['Billiards at Half-Past Nine', 'depicts', 'war']]}

# Evaluation

In [28]:
def create_key(sub_label, rel_label, obj_label):
    # remove spaces and make lower case
    sub_label = re.sub(r"(_|\s+)", '', sub_label).lower()
    rel_label = re.sub(r"(_|\s+)", '', rel_label).lower()
    obj_label = re.sub(r"(_|\s+)", '', obj_label).lower()
    # concatanate them 
    tr_key = f"{sub_label}{rel_label}{obj_label}"
    return tr_key


def calculate_precision_recall_f1(gold, pred):
    if len(pred) == 0:
        return 0, 0, 0
    gold = set(gold)
    pred = set(pred)
    p = len(gold.intersection(pred)) / len(pred)
    r = len(gold.intersection(pred)) / len(gold)
    if p + r > 0:
        f1 = 2 * ((p * r) / (p + r))
    else:
        f1 = 0
    return p, r, f1

In [40]:
for sent_id in list(id_to_gt.keys())[:20]: 
    print(f"Processing: {sent_id}")
    gt_triples = [[tr['sub'], tr['rel'], tr['obj']] for tr in id_to_gt[sent_id]['triples']]
    print(f"  groud truth:")
    for tr in gt_triples:
        print(f"   {tr}")
    if sent_id in id_to_sys:
        system_triples = id_to_sys[sent_id]['triples']
        print(f"  system output: {system_triples}")
        gt_relations = [tr[1].replace(" ", "_") for tr in gt_triples]
        print(f"  gt relations: {gt_relations}")
        filtered_system_triples = [tr for tr in system_triples if tr[1] in gt_relations]
        print(f"  filtered system output:")
        for tr in filtered_system_triples:
            print(f"   {tr}")        
        encoded_system_triples = [create_key(tr[0], tr[1], tr[2]) for tr in filtered_system_triples]
        encoded_gt_triples = [create_key(tr[0], tr[1], tr[2]) for tr in gt_triples]
        precision, recall, f1 = calculate_precision_recall_f1(encoded_gt_triples, encoded_system_triples)
        print(f"precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}")
    print("\n")

Processing: ont_4_book_test_1
  groud truth:
   ['Peter and the Piskies', 'illustrator', 'Raymond Briggs']
   ['Peter and the Piskies', 'publication date', '00  1958']
   ['Peter and the Piskies', 'author', 'Ruth Manning-Sanders']
  system output: [['Peter and the Piskies: Cornish Folk and Fairy Tales', 'illustrator', 'Raymond Briggs']]
  gt relations: ['illustrator', 'publication_date', 'author']
  filtered system output:
   ['Peter and the Piskies: Cornish Folk and Fairy Tales', 'illustrator', 'Raymond Briggs']
precision: 0.0000, recall: 0.0000, f1: 0.0000


Processing: ont_4_book_test_2
  groud truth:
   ['Queen Zixi of Ix', 'illustrator', 'Frederick Richardson']
   ['Queen Zixi of Ix', 'author', 'L. Frank Baum']
  system output: [['Queen Zixi of Ix', 'illustrator', 'Frederick Richardson']]
  gt relations: ['illustrator', 'author']
  filtered system output:
   ['Queen Zixi of Ix', 'illustrator', 'Frederick Richardson']
precision: 1.0000, recall: 0.5000, f1: 0.6667


Processing: ont_